<a href="https://colab.research.google.com/github/T-Sawao/diveintocode-ml/blob/master/term2_sprint10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ベースクラス

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as setattr
from sklearn.metrics import accuracy_score

In [ ]:
class ScratchDeepNeuralNetworkClassifier():
  def __init__(self, bias=1, epoch_num=1, batch_size=20, verbose=True):
        self.bias = bias
        self.epoch = epoch_num
        self.batch_size = batch_size
        self.verbose = verbose

  def fit(self, X, y, X_val=None, y_val=None, sigma=0.01, lr=0.01,  n_nodes1=400, n_nodes2=200, n_output=10):
        self.sigma = sigma
        self.lr = lr
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.n_output = n_output

        self.loss = np.zeros(self.epoch)
        self.val_loss = np.zeros(self.epoch)
        
        # 最適化クラスの呼び出し  
        optimizer = SGD(self.lr)

        # 全結合層にインスタンスを渡す
        self.FC1 = FC(self.n_features, self.n_nodes1, SimpleInitializer(self.sigma), optimizer)
        self.activation1 = Tanh()
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, SimpleInitializer(self.sigma), optimizer)
        self.activation2 = Tanh()
        self.FC3 = FC(self.n_nodes2, self.n_output, SimpleInitializer(self.sigma), optimizer)
        self.activation3 = Softmax()

        # エポック数分の学習
        for i in range(self.epoch):
          # ミニバッチの作成
          get_mini_batch = GetMiniBatch(X, y, self.batch_size)
          # 1エポック（全バッチ）の学習
          for X, Y in get_mini_batch:

            # フォワードプロパゲーションの実行
            A1 = self.FC1.forward(X)
            Z1 = self.activation1.forward(A1)
            A2 = self.FC2.forward(Z1)
            Z2 = self.activation2.forward(A2)
            A3 = self.FC3.forward(Z2)
            Z3 = self.activation3.forward(A3)

            # バックプロパゲーションの実行
            dA3 = self.activation3.backward(Z3, Y) # 交差エントロピー誤差とソフトマックスを合わせている
            dZ2 = self.FC3.backward(dA3)
            dA2 = self.activation2.backward(dZ2)
            dZ1 = self.FC2.backward(dA2)
            dA1 = self.activation1.backward(dZ1)
            dZ0 = self.FC1.backward(dA1) # dZ0は使用しない

          self.loss[i] += loss

          if (type(X_val) != bool):
            self.val = 1
            y_hat_val = self._forward_propagation(x_val)
            loss_val = self._cross_entropy_error(y_hat_val, y_val)
            self.val_loss[i] += loss_val

            # self.acc_val[i] = accuracy_score(np.argmax(y_val, axis=1), np.argmax(y_hat_val, axis=1))
            # verboseをTrueにした際は学習過程を出力
          if self.verbose :
            print(f"--{i+1}回目~loss~-------\n{self.loss[i]}")
            print(f"--{i+1}回目~loss_val~---\n{self.val_loss[i]}")
            # print(f'epoch:{self.epoch:>3} loss:{self.loss:>8,.3f}')

## 問題5（解答）------------------------------------------------------------------------------
    def predict(self, X):
        y_hat = self._forward_propagation(X)
        return np.argmax(y_hat, axis=1)
# 問題7（解答）-------------------------------------------------------------------------------    
    def plot_cost(self):
        plt.title("Num_of_Iteration vs Loss")
        plt.xlabel("Num_of_Iteration")
        plt.ylabel("Loss")
        a = range(self.epoch)
        plt.plot(range(1, self.epoch+1), self.loss, color="b", label="train_loss")
        if self.val ==1:
            plt.plot(range(1, self.epoch+1), self.val_loss, color="orange", label="val_loss")
        plt.grid()
        plt.legend()

## 全結合層のクラス

In [ ]:
class FC:
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.W = initializer.W(self.n_nodes1, self.n_nodes2)
        self.B = initializer.B(1, self.n_nodes1)
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する

    # フォワードプロパゲーション時の処理
    def forward(self, X):
      out = X@self.W+self.B
      return out
    
    # バックプロパゲーション時の処理
    def backward(self, dA):
      dZ = dA@(self.w.T)
      dW = (dz.T)@dA
      dB = np.sum(dA, axis=0)
      
      # 更新
      self.W = self.optimizer.update(dW, self.W)
      self.B = self.optimizer.update(dB, self.B)
      return dZ

### 初期化クラス

In [ ]:
# SimpleInitializer ------------------------------------------------------------
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma

    def W(self, n_nodes1, n_nodes2):
        W = sigma * np.random.randn(n_nodes1, n_nodes2)
        return W

    def B(self, n_nodes2):
        B = sigma * np.random.randn(1, n_nodes2)
        return B

# XavierInitializer ------------------------------------------------------------
class XavierInitializer:
    def __init__(self, Xavier):
        self.Xavier = Xavier

    def W(self, n_nodes1, n_nodes2):
        W = np.random.randn(n_nodes1, n_nodes2) * np.sqrt(1.0 / n_nodes1)
        return W

    def B(self, n_nodes2):
        B = np.random.randn(1, n_nodes2) * np.sqrt(1.0 / 1.0)
        return B

# He-----------------------------------------------------------------------------
class HeInitializer:
    def __init__(self, He):
        self.He = He

    def W(self, n_nodes1, n_nodes2):
        W = p.random.randn(n_nodes1, n_nodes2) * np.sqrt(2.0 / n_nodes1)
        return W

    def B(self, n_nodes2):
        B = p.random.randn(1, n_nodes2) * np.sqrt(2.0 / 1)
        return B

## 最適化クラス

In [ ]:
# SGD---------------------------------------------
class SGD:
    def __init__(self, lr):
        self.lr = lr
        
    def update(self, dWorB, WorB):
      self.WorB = WorB
      self.WorB -= self.lr*dWorB
      return self.WorB

# AdaGrad----------------------------------------
class  AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None

    def update(self, dWorB, WorB):
      self.WorB = WorB
      self.WorB -= self.lr*dWorB
      
      if self.h is None:
        self.h = {}
        for key, val in WorB.items():
            self.h[key] = np.zeros_like(val)

      for key in WorB.keys():
          self.h[key] += dWorB[key] * dWorB[key]
          WorB[key] -= self.lr * dWorB[key] / (np.sqrt(self.h[key]) + 1e-7)
      return self.WorB

### 活性化関数クラス

In [ ]:
# ソフトマックス関数のクラス----------------------------------------------
class Softmax:
  def __init__(self):

    # forward時の処理
    def forward(self, X):
      y_hat = np.exp(X) / np.sum(np.exp(X), axis = 1).reshape(-1,1)
      return y_hat

    # backward時の処理
    def backward(self, X, Y):
      loss = self._cross_entropy_error(self, X, Y)
      dA3 = X -Y
      return dA3

    # 交差エントロピー誤差の関数
    def _cross_entropy_error(self, X, Y):
      return -np.mean(Y * np.log(X + 1e-7))

# ReLU関数のクラス----------------------------------------------
class ReLU:
  def __init__(self):

    # forward時の処理
    def forward(self, X):
      return np.maximum(0, X)

    # backward時の処理
    def backward(self, X, Y):
      loss = self._cross_entropy_error(self, X, Y)
      return np.maximum(0, X)

    # 交差エントロピー誤差の関数
    def _cross_entropy_error(self, X, Y):
      return -np.mean(Y * np.log(X + 1e-7))

## 前処理

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 平滑化（flatten）
X_train_fltten = X_train.reshape(-1, 784)
X_test_fltten = X_test.reshape(-1, 784)

# float化と0or1処理
X_train_flt = X_train_fltten.astype(np.float)
X_test_flt = X_test_fltten.astype(np.float)
X_train_flt /= 255
X_test_flt /= 255

# one hot処理
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

In [ ]:
# train, testの分割
from sklearn.model_selection import train_test_split
x_train, x_val, y_train_one_hot, y_val_one_hot = train_test_split(X_train_flt, y_train_one_hot, test_size=0.2)
print("x_train",x_train.shape, "x_val", x_val.shape, "y_train_one_hot.shape", y_train_one_hot.shape, "y_val_one_hot", y_val_one_hot.shape) # (48000, 784)